In [4]:
# !pip install -q --upgrade clickzetta_zettapark_python -i https://pypi.tuna.tsinghua.edu.cn/simple

In [1]:
from clickzetta.zettapark.session import Session
import json
# 从配置文件中读取参数
with open('security/config-uat.json', 'r') as config_file:
    config = json.load(config_file)

# 创建会话
session = Session.builder.configs(config).create()

In [2]:
session.sql("delete volume zettapark_dev_20240529171730 subdirectory '/'").show()

---------------------
|result_message     |
---------------------
|OPERATION SUCCEED  |
---------------------



In [3]:
df1 = session.create_dataframe([[1,2],[3,4]], schema=["a", "b"])
df1.show()

---------
|a  |b  |
---------
|1  |2  |
|3  |4  |
---------



In [4]:
unload_path = "volume://zettapark_dev_20240529171730/unload_0606/"
df1.write.copy_into_volume(unload_path, file_format_type="csv")

[Row(result_message='OPERATION SUCCEED')]

In [5]:
session.sql("show volume directory zettapark_dev_20240529171730").show()

---------------------------------------------------------------------------------------------------------------------
|relative_path              |url                                                 |size  |last_modified_time         |
---------------------------------------------------------------------------------------------------------------------
|unload_0606/part00001.csv  |oss://czsampledatahz/zettapark_dev_202405291717...  |8     |2024-12-24 15:46:22+08:00  |
---------------------------------------------------------------------------------------------------------------------



In [6]:
from clickzetta.zettapark.types import StructType, StructField, LongType, StringType
schema = StructType([StructField("x", LongType()), StructField("y", StringType())])
path = "volume://zettapark_dev_20240529171730/unload_0606/"
csv_loaded = session.read.option("FORMAT_NAME", "csv").schema(schema).csv(path)
csv_loaded.show()

---------
|x  |y  |
---------
|1  |2  |
|3  |4  |
---------



In [7]:
session.close()